## Import relevent packages

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from dateutil import parser

## Clean the medicare hospital general information data set

The overall rating includes 57 of the more than 100 measures reported on Hospital Compare, divided into seven measure groups or categories: mortality, safety of care, readmission, patient experience, effectiveness of care, timeliness of care, and efficient use of medical imaging. The table below lists the measures included in the Hospital Compare overall rating. This list includes all possible measures that could be included. Once reporting thresholds are met, a hospital’s overall rating is calculated using only those measures for which data are available. 

Measure Category and Weight Used in Calculation
* Mortality	22%
* Safety	22%
* Readmission	22%
* Patient Experience	22%
* Effectiveness of Care	4%
* Timeliness of Care	4%
* Efficient Use of Medical Imaging	4%

In [4]:
general_raw = 'Hospital_General_Information.csv'

In [5]:
g = pd.read_csv(general_raw)

## Tidy columns

In [6]:
g.columns = g.columns.str.replace(' ', '_')

In [7]:
g.head(0)

,Provider_ID,Hospital_Name,Address,City,State,ZIP_Code,County_Name,Phone_Number,Hospital_Type,Hospital_Ownership,...,Readmission_national_comparison_footnote,Patient_experience_national_comparison,Patient_experience_national_comparison_footnote,Effectiveness_of_care_national_comparison,Effectiveness_of_care_national_comparison_footnote,Timeliness_of_care_national_comparison,Timeliness_of_care_national_comparison_footnote,Efficient_use_of_medical_imaging_national_comparison,Efficient_use_of_medical_imaging_national_comparison_footnote,Location


In [8]:
g = g[['Hospital_Name','State', 'ZIP_Code', 'Provider_ID', 'Hospital_Type', 'Hospital_Ownership', 'Emergency_Services', 'Meets_criteria_for_meaningful_use_of_EHRs', 'Hospital_overall_rating','Safety_of_care_national_comparison', 'Patient_experience_national_comparison', 'Effectiveness_of_care_national_comparison','Timeliness_of_care_national_comparison', 'Efficient_use_of_medical_imaging_national_comparison']]

In [9]:
g = g.rename(columns = lambda x: x + '_g')

In [10]:
g.rename(columns = {'Meets_criteria_for_meaningful_use_of_EHRs_g': 'EHRs_g', 'Hospital_Type_g':'Type','Hospital_Ownership_g':'Ownership_g'}, inplace = True)

In [11]:
g.columns = g.columns.str.replace('_national_comparison', '')

## Tidy nulls

In [12]:
g = g.replace('Not Available', np.nan)

## Save cleaned general hospital info to CSV

In [13]:
g.to_csv('general_clean_save.csv')

In [14]:
g.Type.unique()

array(['Acute Care Hospitals', 'Critical Access Hospitals', 'Childrens'], dtype=object)

## MSPB medicare spending per benificiary

Gives spending by provider information:

The "Medicare hospital spending per patient (Medicare Spending per Beneficiary)" measure shows whether Medicare spends more, less or about the same per Medicare patient treated in a specific hospital, compared to how much Medicare spends per patient nationally. This measure includes any Medicare Part A and Part B payments made for services provided to a patient during the 3 days prior to the hospital stay, during the stay, and during the 30 days after discharge from the hospital. 

In [15]:
mspb = pd.read_csv("Medicare_hospital_spending_per_patient__Medicare_Spending_per_Beneficiary____Additional_Decimal_Places.csv")

In [16]:
#test_index_compatibility = mspb['Provider_ID'].isin(general['Provider_ID_g'])

In [17]:
#test_index_compatibility.describe()

## Tidy column names

In [18]:
mspb = mspb.rename(columns=lambda x: x + '_m')

In [19]:
mspb.rename(columns = {'Value_m': 'MSPB'}, inplace = True)

## Drop unecessary columns

In [20]:
mspb.drop(['Measure_ID_m', 'Footnote_m', 'Start_Date_m', 'End_Date_m'], inplace = True, axis = 1)

## Convert to numeric

In [21]:
mspb['MSPB'] = mspb['MSPB'].apply(pd.to_numeric, errors='coerce')


## Save cleaned MSPB to CSV

In [22]:
mspb.to_csv('mspb_clean.csv')

## Clean medicare readmissions data set

In October 2012, CMS began reducing Medicare payments for Inpatient Prospective Payment System hospitals with excess readmissions. Excess readmissions are measured by a ratio, by dividing a hospital’s number of “predicted” 30-day readmissions for heart attack, heart failure, and pneumonia by the number that would be “expected,” based on an average hospital with similar patients. A ratio greater than 1 indicates excess readmissions.

In [23]:
readmissions_raw = 'Hospital_Readmissions_Reduction_Program.csv'

In [24]:
r = pd.read_csv(readmissions_raw)

## Tidy up column names

In [25]:
r.columns = r.columns.str.replace(' ', '_')

In [26]:
r = r[['Hospital_Name', 'Measure_Name', 'Number_of_Discharges', 'Excess_Readmission_Ratio','Predicted_Readmission_Rate', 'Expected_Readmission_Rate', 'Number_of_Readmissions']] 

In [27]:
r = r.rename(columns=lambda x: x + '_r')

## Clean nulls

In [28]:
r = r.replace('Not Available', np.nan)

## Convert to numeric

In [29]:
r.iloc[:, 2:7] = r.iloc[:, 2:7].apply(pd.to_numeric, errors='coerce')

## Simplify Readmission Code information

In [30]:
r = r.replace('READM-30-AMI-HRRP', 'AMI').replace('READM-30-CABG-HRRP','CABG').replace('READM-30-COPD-HRRP','COPD').replace('READM-30-HF-HRRP', 'HF').replace('READM-30-HIP-KNEE-HRRP', 'HIP-KNEE').replace('READM-30-PN-HRRP', 'PN')

In [31]:
r.Measure_Name_r.unique()

array(['AMI', 'CABG', 'COPD', 'HF', 'HIP-KNEE', 'PN'], dtype=object)

## Save to Readmission info to CSV

In [32]:
r.head(2)

,Hospital_Name_r,Measure_Name_r,Number_of_Discharges_r,Excess_Readmission_Ratio_r,Predicted_Readmission_Rate_r,Expected_Readmission_Rate_r,Number_of_Readmissions_r
0,SOUTHEAST ALABAMA MEDICAL CENTER,AMI,781.0,0.9837,15.3580,15.6121,119.0
1,SOUTHEAST ALABAMA MEDICAL CENTER,CABG,273.0,1.0618,13.8887,13.0809,40.0


In [33]:
r.to_csv('readmissions_clean_save.csv')

## Create a readmissions dataset using average readmissions data across measures. This will facilitate merges with other data frames

In [34]:
r_means = r.groupby(['Hospital_Name_r']).mean()

In [35]:
r_means.reset_index(inplace = True)

In [36]:
r_means.to_csv('readmissions_clean_mean_save.csv')

## Merge General Hospital and Readmissions Data Sets

In [63]:
r_g = pd.merge(left = g, right = r_means, how = 'outer', left_on = 'Hospital_Name_g', right_on = 'Hospital_Name_r')

## Now merge with MSPB Data Set

In [64]:
r_g_mspb = pd.merge(left = r_g, right = mspb, how = 'outer', left_on = 'Provider_ID_g', right_on = 'Provider_ID_m')

## Drop redundant columns

In [65]:
r_g_mspb.head(1)

,Hospital_Name_g,State_g,ZIP_Code_g,Provider_ID_g,Type,Ownership_g,Emergency_Services_g,EHRs_g,Hospital_overall_rating_g,Safety_of_care_g,...,Timeliness_of_care_g,Efficient_use_of_medical_imaging_g,Hospital_Name_r,Number_of_Discharges_r,Excess_Readmission_Ratio_r,Predicted_Readmission_Rate_r,Expected_Readmission_Rate_r,Number_of_Readmissions_r,Provider_ID_m,MSPB
0,SOUTHEAST ALABAMA MEDICAL CENTER,AL,36301.0,10001.0,Acute Care Hospitals,Government - Hospital District or Authority,True,True,3,Above the National average,...,Same as the National average,Same as the National average,SOUTHEAST ALABAMA MEDICAL CENTER,624.166667,1.041817,15.352883,14.956117,106.333333,10001.0,0.999877


In [66]:
r_g_mspb.drop(['Provider_ID_g','Hospital_Name_r', 'Provider_ID_m'], inplace = True, axis = 1)

## Generate Time Zone Data Frame

In [67]:
state_to_region = { 'AK': 'US/Alaska', 'AL': 'US/Central', 'AR': 'US/Central', 'AS': 'US/Samoa', 'AZ': 'US/Mountain', 'CA': 'US/Pacific', 'CO': 'US/Mountain', 'CT': 'US/Eastern', 'DC': 'US/Eastern', 'DE': 'US/Eastern', 'FL': 'US/Eastern', 'GA': 'US/Eastern', 'GU': 'Pacific/Guam', 'HI': 'US/Hawaii', 'IA': 'US/Central', 'ID': 'US/Mountain', 'IL': 'US/Central', 'IN': 'US/Eastern', 'KS': 'US/Central', 'KY': 'US/Eastern', 'LA': 'US/Central', 'MA': 'US/Eastern', 'MD': 'US/Eastern', 'ME': 'US/Eastern', 'MI': 'US/Eastern', 'MN': 'US/Central', 'MO': 'US/Central', 'MP': 'Pacific/Guam', 'MS': 'US/Central', 'MT': 'US/Mountain', 'NC': 'US/Eastern', 'ND': 'US/Central', 'NE': 'US/Central', 'NH': 'US/Eastern', 'NJ': 'US/Eastern', 'NM': 'US/Mountain', 'NV': 'US/Pacific', 'NY': 'US/Eastern', 'OH': 'US/Eastern', 'OK': 'US/Central', 'OR': 'US/Pacific', 'PA': 'US/Eastern', 'PR': 'America/Puerto_Rico', 'RI': 'US/Eastern', 'SC': 'US/Eastern', 'SD': 'US/Central', 'TN': 'US/Central', 'TX': 'US/Central', 'UT': 'US/Mountain', 'VA': 'US/Eastern', 'VI': 'America/Virgin', 'VT': 'US/Eastern', 'WA': 'US/Pacific', 'WI': 'US/Central', 'WV': 'US/Eastern', 'WY': 'US/Mountain', '' : 'US/Pacific', '--': 'US/Pacific' }

In [68]:
state_to_region_df = pd.DataFrame(state_to_region.items())

In [69]:
state_to_region_df.columns = ['State_name', 'Region']

## Merge Data Frame with Time Zone Information

In [70]:
r_g_mspb_geo = pd.merge(left = r_g_mspb , right = state_to_region_df, how = 'left', left_on = 'State_g', right_on='State_name')

In [71]:
r_g_mspb_geo.drop(['State_g'], axis = 1, inplace = True)

In [72]:
r_g_mspb_geo['Emergency_Services_g'] = r_g_mspb_geo['Emergency_Services_g'].astype('str')

In [73]:
r_g_mspb_geo.to_csv('r_g_mspb.csv')

In [74]:
r_g_mspb_geo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4825 entries, 0 to 4824
Data columns (total 20 columns):
Hospital_Name_g                       4807 non-null object
ZIP_Code_g                            4807 non-null float64
Type                                  4807 non-null object
Ownership_g                           4807 non-null object
Emergency_Services_g                  4825 non-null object
EHRs_g                                4363 non-null object
Hospital_overall_rating_g             3584 non-null object
Safety_of_care_g                      2654 non-null object
Patient_experience_g                  3454 non-null object
Effectiveness_of_care_g               3627 non-null object
Timeliness_of_care_g                  3565 non-null object
Efficient_use_of_medical_imaging_g    2790 non-null object
Number_of_Discharges_r                3031 non-null float64
Excess_Readmission_Ratio_r            3156 non-null float64
Predicted_Readmission_Rate_r          3156 non-null float64
Expe

In [75]:
r_g_mspb_geo.Type.describe

<bound method Series.describe of 0            Acute Care Hospitals
1            Acute Care Hospitals
2            Acute Care Hospitals
3            Acute Care Hospitals
4            Acute Care Hospitals
5            Acute Care Hospitals
6            Acute Care Hospitals
7            Acute Care Hospitals
8            Acute Care Hospitals
9            Acute Care Hospitals
10           Acute Care Hospitals
11           Acute Care Hospitals
12           Acute Care Hospitals
13           Acute Care Hospitals
14           Acute Care Hospitals
15           Acute Care Hospitals
16           Acute Care Hospitals
17           Acute Care Hospitals
18           Acute Care Hospitals
19           Acute Care Hospitals
20           Acute Care Hospitals
21           Acute Care Hospitals
22           Acute Care Hospitals
23           Acute Care Hospitals
24           Acute Care Hospitals
25           Acute Care Hospitals
26           Acute Care Hospitals
27      Critical Access Hospitals
28           Ac